In [34]:
from datetime import datetime
import os
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek
import pyspark.sql.types as T    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
input_data = "s3a://udacity-dend/"
output_data = "s3://sparkifydatasource/sparkify_test/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
song_schema = T.StructType([T.StructField('artist_id'       , T.StringType())
                             , T.StructField('artist_latitude'  , T.StringType())
                             , T.StructField('artist_location'  , T.StringType())
                             , T.StructField('artist_longitude' , T.StringType())
                             , T.StructField('artist_name'      , T.StringType())
                             , T.StructField('duration'         , T.DoubleType())
                             , T.StructField('num_songs'        , T.IntegerType())
                             , T.StructField('song_id'          , T.StringType())
                             , T.StructField('title'            , T.StringType())
                             , T.StructField('year'             , T.IntegerType())
                              ])
# Log table schema for JSON read
log_schema = T.StructType([ T.StructField('artist'          , T.StringType())
                              , T.StructField('auth'            , T.StringType())
                              , T.StructField('gender'          , T.StringType())
                              , T.StructField('lastName'        , T.StringType())
                              , T.StructField('length'          , T.DoubleType())
                              , T.StructField('level'           , T.StringType())
                              , T.StructField('firstName'       , T.StringType())
                              , T.StructField('itemInSession'   , T.IntegerType())
                              , T.StructField('location'        , T.StringType())
                              , T.StructField('method'          , T.StringType())
                              , T.StructField('page'            , T.StringType())
                              , T.StructField('registration'    , T.DoubleType())
                              , T.StructField('sessionId'       , T.IntegerType())
                              , T.StructField('song'            , T.StringType())
                              , T.StructField('status'          , T.IntegerType())
                              , T.StructField('ts'              , T.LongType())
                              , T.StructField('userAgent'       , T.StringType())
                              , T.StructField('userId'          , T.StringType())
                              ])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
song_data = os.path.join(input_data, 'song_data/*/*/*/*.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df = spark.read.json(song_data, schema=song_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: integer (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)

In [13]:
# extract columns to create songs table
songs_table = df.select('song_id', 'title', 'artist_id', 'year', 'duration')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
type(songs_table)
songs_table.printSchema()
songs_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- duration: double (nullable = true)

+------------------+--------------------+------------------+----+----------+
|           song_id|               title|         artist_id|year|  duration|
+------------------+--------------------+------------------+----+----------+
|SOVIYJY12AF72A4B00|The Dead Next Doo...|AR4T2IF1187B9ADBB7|1983| 233.22077|
|SOVYXYL12AF72A3373|Rebel Yell (1999 ...|AR4T2IF1187B9ADBB7|1983| 287.92118|
|SOEPTVC12A67ADD0DA|To Zucchabar ["Gl...|ARQ846I1187B9A7083|   0| 196.04853|
|SOLQYSZ12AB0181F97|    Mony Mony (Live)|AR4T2IF1187B9ADBB7|1987| 247.53587|
|SOVPFJK12A6701CB16|Barcelona - (Frie...|AR3TZ691187FB3DBB1|2000| 273.44934|
|SOETDBF12A81C20BC0|       Body Snatcher|AR4T2IF1187B9ADBB7|2005| 237.06077|
|SOIEXLS12A6D4F792F|           Shangrila|AR4T2IF1187B9ADBB7|1993| 443.14077|
|SOQEBML12A8C136AA4|Werther (1

In [17]:
songs_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14896

In [19]:
song_out_path = os.path.join(output_data, 'sparkify_songs_table/')
songs_table.write.parquet(song_out_path, mode='overwrite', partitionBy=('year','artist_id'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# extract columns to create artists table
artists_table = df.select('artist_id', \
                              col('artist_name').alias('name'), \
                              col('artist_location').alias('location'),\
                              col('artist_latitude').alias('latitude'),\
                              col('artist_longitude').alias('longitude'))\
                              .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
artists_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)

In [22]:
 # write artists table to parquet files
artist_out_path = os.path.join(output_data, 'sparkify_artist_table/')
artists_table.write.parquet(artist_out_path, mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
# get filepath to log data file
log_data = os.path.join(input_data, 'log_data/*/*/*.json')
df = spark.read.json(log_data, schema=log_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
# filter by actions for song plays
df = df.filter(df.page == 'NextSong')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+------+--------+---------+-----+---------+-------------+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|gender|lastName|   length|level|firstName|itemInSession|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+------+--------+---------+-----+---------+-------------+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|            Harmonia|Logged In|     M|   Smith|655.77751| free|     Ryan|            0|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|         The Prodigy|Logged In|     M|   Smith|260.07465| free|     Ryan|            1|San Jose-Sunnyva

In [60]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6820

In [77]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- itemInSession: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: integer (nullable = true)
 |-- song: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [78]:
# extract columns for users table
users_table = df.repartition(df.userId) \
                    .select(col('userId').alias('user_id'), \
                            col('firstName').alias('first_name'),\
                            col('lastName').alias('last_name'), \
                            'gender',
                            'level') \
                    .where(col("user_id").isNotNull()) \
                    .dropDuplicates()

# write users table to parquet files
user_out_path = os.path.join(output_data, 'sparkify_user_table/')
users_table.write.parquet(user_out_path, mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
users_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

In [79]:
# Defining User Defined Function (UDF) to convert log timestamp (seconds since epoch) to \
    
get_timestamp = udf(lambda x : datetime.utcfromtimestamp(int(x)/1000), T.TimestampType())
df = df.withColumn('timestamp', get_timestamp(df.ts))

get_datetime = udf(lambda x : datetime.utcfromtimestamp(int(x)/1000), T.DateType())
df = df.withColumn("datetime", get_timestamp("ts"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
# extract columns to create time table
time_table = df.select(col('timestamp').alias('start_time') \
                  , hour('timestamp').alias('hour') \
                  , dayofmonth('timestamp').alias('day') \
                  , weekofyear('timestamp').alias('week') \
                  , month('timestamp').alias('month') \
                  , year('timestamp').alias('year') \
                  , dayofweek('timestamp').alias('weekday'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- itemInSession: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: integer (nullable = true)
 |-- song: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- datetime: timestamp (nullable = true)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
time_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

In [83]:
# write time table to parquet files partitioned by year and month
time_out_path = os.path.join(output_data, 'sparkify_time_table/')
time_table.write.parquet(time_out_path, mode='overwrite', partitionBy=('year', 'month'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
# read in song data to use for songplays table
song_data = os.path.join(input_data, 'song_data/*/*/*/*.json')
song_df = spark.read.json(song_data, schema=song_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
# extract columns from joined song and log datasets to create songplays table
df_song_log = df.join(song_df, [df.song == song_df.title,\
                                        df.artist == song_df.artist_name])
songplays_table = df_song_log.select(col('timestamp').alias('start_time'),\
                                        col('userId').alias('user_id'),\
                                        'level',\
                                        'song_id',\
                                        'artist_id',\
                                        col('sessionId').alias('session_id'),\
                                        'location',\
                                        col('userAgent').alias('user_agent'), \
                                        year(df.timestamp).alias('year'), \
                                        month(df.timestamp).alias('month'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
songplays_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

In [88]:
songplays_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
|          start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|year|month|
+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
|2018-11-21 08:25:...|     88| paid|SOCHPTV12A6BD53113|ARN8NCB1187FB49652|       744|Sacramento--Rosev...|"Mozilla/5.0 (Mac...|2018|   11|
|2018-11-29 16:58:...|     49| paid|SOGXSWA12A6D4FBC99|ARPFHN61187FB575F6|      1041|San Francisco-Oak...|Mozilla/5.0 (Wind...|2018|   11|
|2018-11-28 08:18:...|     58| paid|SOJWCWM12A8C13B664|ARM6T8I1187FB36CC8|       887|Augusta-Richmond ...|"Mozilla/5.0 (Win...|2018|   11|
|2018-11-28 23:34:...|     24| paid|SOHRHCN12AB018B0F4|ARHQBRZ1187FB3BDA2|       984|Lake Havasu City-...|"Mozilla/5.0 (Win...|2018|   11|
|2018-11-26 15:37:...|     

In [89]:
# write songplays table to parquet files partitioned by year and month
songplays_out_path = os.path.join(output_data, 'sparkify_songplays_table/')
songplays_table.write.parquet(songplays_out_path, mode='overwrite', partitionBy=('year', 'month'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…